In [1]:
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GaussianNoise
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras.utils import vis_utils as vizu
from keras.layers import Dropout
from keras import callbacks
from keras import backend
from scipy.integrate import simps
from scipy.stats import f
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, mutual_info_regression
def knn_baseline_model():
    
    def root_mean_squared_error(y_true, y_pred):
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
    # create model
    model = Sequential()
    #model.add(GaussianNoise(stddev=0.1, input_shape=(14,)))
    model.add(Dense(7, activation='relu', input_shape=(18,)))
    model.add(Dropout(0.5))
    model.add(GaussianNoise(0.1))
    model.add(Dense(24, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss=root_mean_squared_error, optimizer='adam')
    #model.compile(loss='mse', optimizer='adam')
    return model
def coeff_determination(y_true, y_pred):
    ssres = np.sum(np.square(y_true - y_pred))
    sstot = np.sum(np.square(y_true - np.mean(y_pred)))
    return 1 - ssres / sstot
def REC(y_true , y_pred):
    
    Accuracy = []
    Begin_Range = 0
    End_Range = 1.5
    Interval_Size = 0.01
    
    # List of epsilons
    Epsilon = np.arange(Begin_Range , End_Range , Interval_Size)
    
    # Main Loops
    for i in range(len(Epsilon)):
        count = 0.0
        for j in range(len(y_true)):
            y_true[j]=float(y_true[j])
            y_true[j]
            y_pred[j]
            
            np.linalg.norm(y_true[j] - y_pred[j])
            np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]
            
            if np.linalg.norm(y_true[j] - y_pred[j]) / np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]:
                count = count + 1
        
        Accuracy.append(count/len(y_true))
    
    # Calculating Area Under Curve using Simpson's rule
    AUC = simps(Accuracy , Epsilon ) / End_Range
        
    # returning epsilon , accuracy , area under curve    
    return Epsilon, Accuracy, AUC
def model_evaluate (model, X_train, X_test, y_train, y_test):

    y_pred = model.predict(X_test)
    Deviation, Accuracy, auc = REC(y_test.values, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    #r2 = r2_score(y_test, y_pred)
    r2 = coeff_determination(y_test, y_pred)
    mape = ((y_test - y_pred) / y_test).abs().mean()*100
    
    f_test = np.var(y_test) / np.var(y_pred)
    df1 = len(y_test) - 1
    df2 = len(y_pred) - 1
    p_value = 1 - 2 * abs(0.5 - f.cdf(f_test, df1, df2))
    
    p_stars = ''
    if p_value <= 0.05:
        p_stars = '*'
    if p_value <= 0.01:
        p_stars = '**'
    if p_value <= 0.001:
        p_stars = '***'
    #print(F, p_value)
    
    y_validate = model.predict(X_train)
    training_loss = sqrt(mean_squared_error(y_train, y_validate))
    
    return_dict = {'rmse':rmse, 
                   'mape':mape, 
                   'r2':r2, 
                   'auc':auc,
                   'training_loss':training_loss, 
                   'f_test':f_test,
                   'p':p_stars,
                   'y_pred':y_pred, 
                   'y_validate':y_validate}
    
    return return_dict

conn = MongoClient('140.115.53.147', 27017)
db = conn['NCU_CHKB']
collection = db['chkb_all_features_by_week']
all_features = pd.DataFrame(list(collection.find()))
all_features = all_features.drop(['_id'], axis=1)
collection = db['chkb_final_score']
final_score = pd.DataFrame(list(collection.find()))
final_score = final_score.drop(['_id'], axis=1)
all_features.head()

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


,Cumulative_num_days,Problem_num_day,active_num_days,active_sum_count,homework_complete_num,homework_num,num_complete,num_watched,pause_video_sum,seek_video_sum,stop_video_sum,username,video_backward_seek_sum,video_forward_seek_sum,video_num_days,video_pause_sum,video_stop_sum,video_sum_count,week
0,0,0,0,0,0,0,0,0,0,0,0,Bryan,0,0,0,0,0,0,3
1,0,0,0,0,0,0,0,0,0,0,0,Y1050185,0,0,0,0,0,0,2
2,4,3,0,0,3,18,0,0,0,0,0,Y1050185,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,0,0,0,Y1050185,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,Y1050185,0,0,0,0,0,0,6


In [2]:
chkb = pd.merge(all_features, final_score,on='username')
chkb.fillna(0.0, inplace=True)
chkb.head()

,Cumulative_num_days,Problem_num_day,active_num_days,active_sum_count,homework_complete_num,homework_num,num_complete,num_watched,pause_video_sum,seek_video_sum,stop_video_sum,username,video_backward_seek_sum,video_forward_seek_sum,video_num_days,video_pause_sum,video_stop_sum,video_sum_count,week,final_score
0,3,2,2,245,3,3,5,6,6,0,0,Y1050274,0,0,1,7,0,7,1,80
1,3,1,3,1011,3,3,13,16,16,10,0,Y1050274,10,22,3,42,0,79,3,80
2,0,0,1,464,0,0,5,7,7,2,0,Y1050274,4,22,1,14,0,42,4,80
3,3,1,4,1159,3,3,25,33,32,11,1,Y1050274,21,15,3,161,1,206,5,80
4,3,1,4,314,3,3,3,4,4,2,0,Y1050274,1,3,1,20,0,24,2,80


In [3]:

chkb[chkb.columns.difference(['username'])]=chkb[chkb.columns.difference(['username'])].astype(float)
data=chkb.groupby('username').sum().div(6)


In [4]:
data

,Cumulative_num_days,Problem_num_day,active_num_days,active_sum_count,homework_complete_num,homework_num,num_complete,num_watched,pause_video_sum,seek_video_sum,stop_video_sum,video_backward_seek_sum,video_forward_seek_sum,video_num_days,video_pause_sum,video_stop_sum,video_sum_count,week,final_score
username,,,,,,,,,,,,,,,,,,,
,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
Y1050274,2.166667,1.000000,2.666667,593.000000,2.166667,2.166667,9.833333,12.666667,12.500000,5.333333,0.166667,8.166667,14.000000,1.833333,46.666667,0.166667,72.500000,3.5,80.000000
Y1050275,0.500000,0.166667,1.333333,325.666667,0.500000,0.500000,5.833333,7.000000,6.500000,4.500000,0.500000,14.333333,5.333333,1.000000,30.000000,0.500000,57.666667,3.5,20.000000
Y1050276,2.166667,1.333333,2.333333,710.333333,2.166667,2.166667,6.833333,9.500000,9.000000,2.000000,0.666667,3.000000,2.000000,1.500000,36.166667,0.666667,45.000000,3.5,40.000000
Y1050277,0.000000,0.000000,0.666667,441.000000,0.000000,0.000000,4.000000,4.500000,4.500000,2.666667,0.666667,9.166667,4.000000,0.666667,40.333333,0.666667,59.166667,3.5,10.000000
Y1050278,2.166667,1.500000,3.333333,893.666667,2.166667,2.166667,3.500000,5.666667,4.666667,2.166667,1.000000,4.166667,1.000000,2.166667,10.833333,1.000000,20.666667,3.5,54.000000
Y1050279,2.000000,1.500000,4.500000,1833.166667,2.000000,2.000000,10.666667,13.333333,13.166667,6.333333,1.000000,13.666667,10.000000,3.333333,129.833333,1.000000,174.166667,3.5,70.000000
Y1050280,0.333333,0.333333,2.166667,115.500000,0.333333,0.333333,2.500000,3.500000,3.166667,0.500000,1.000000,0.500000,0.833333,1.500000,8.500000,1.000000,12.833333,3.5,10.000000
Y1050281,0.000000,0.000000,0.333333,43.000000,0.000000,0.000000,0.166667,0.500000,0.500000,0.500000,0.166667,0.666667,0.500000,0.166667,4.833333,0.166667,6.666667,3.5,10.000000


In [5]:
data.to_csv('data.csv')
data_y= data['final_score']
data_x = data.drop(['final_score'], axis=1)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3)

In [7]:
mlp = KerasRegressor(build_fn=knn_baseline_model, epochs=1000, batch_size=2, verbose=0)

In [8]:
mlp.fit(X_train, y_train) 
result_dict = model_evaluate (mlp, X_train, X_test, y_train, y_test)
print(result_dict)

{'rmse': 22.303835559264794, 'mape': inf, 'r2': 0.36343627956314695, 'auc': 0.7383333333333333, 'training_loss': 20.829035674471598, 'f_test': 7.27706325756701, 'p': '***', 'y_pred': array([25.870998 , 12.457593 , 27.370667 , 11.143935 , 14.65972  ,
       34.528393 , 32.447624 , 11.730806 , 25.738949 , 29.6389   ,
       27.977821 , 28.12004  ,  9.872868 , 12.737541 ,  8.716924 ,
       20.261917 , 19.928217 ,  8.407033 ,  8.716642 , 10.989126 ,
       22.166124 , 14.509749 , 12.690058 , 22.38848  ,  3.571225 ,
       11.6981325, 10.326227 ,  8.603278 , 34.912106 , 16.140915 ,
       31.132872 ,  3.571225 ,  8.673273 ,  9.959598 ,  8.530713 ,
       35.601437 ,  3.571225 , 15.286592 , 30.539928 , 34.16169  ,
        8.549713 ,  9.166967 , 14.554719 , 14.836762 , 17.155346 ,
       10.972357 ,  9.062945 , 13.088229 , 26.356728 , 13.463099 ,
       12.510607 , 11.488221 , 17.350956 , 20.552513 , 11.230292 ,
        9.681004 ], dtype=float32), 'y_validate': array([37.041767 , 38.429348 ,